## Resilient Distributed Datasets (RDDs) - Lab

Resilient Distributed Datasets (RDD) are fundamental data structures of Spark. An RDD is essentially the Spark representation of a set of data, spread across multiple machines, with APIs to let you act on it. An RDD can come from any data source, e.g. text files, a database, a JSON file, etc.


## Objectives

You will be able to:

- Apply the map(func) transformation to a given function on all elements of an RDD in different partitions 
- Apply a map transformation for all elements of an RDD 
- Compare the difference between a transformation and an action within RDDs 
- Use collect(), count(), and take() actions to trigger spark transformations  
- Use filter to select data that meets certain specifications within an RDD 
- Set number of partitions for parallelizing RDDs 
- Create RDDs from Python collections 


## What are RDDs? 

To get a better understanding of RDDs, let's break down each one of the components of the acronym RDD:

Resilient: RDDs are considered "resilient" because they have built-in fault tolerance. This means that even if one of the nodes goes offline, RDDs will be able to restore the data. This is already a huge advantage compared to standard storage. If a standard computer dies while performing an operation, all of its memory will be lost in the process. With RDDs, multiple nodes can go offline, and the action will still be held in working memory.

Distributed: The data is contained on multiple nodes of a cluster-computing operation. It is efficiently partitioned to allow for parallelism.

Dataset: The dataset has been * partitioned * across the multiple nodes. 

RDDs are the building block upon which more high-level Spark operations are based upon. Chances are, if you are performing an action using Spark, the operation involves RDDs. 



Key Characteristics of RDDs:

- Immutable: Once an RDD is created, it cannot be modified. 
- Lazily Evaluated: RDDs will not be evaluated until an action is triggered. Essentially, when RDDs are created, they are programmed to perform some action, but that function will not get activated until it is explicitly called. The reason for lazy evaluation is that allows users to organize the actions of their Spark program into smaller actions. It also saves unnecessary computation and memory load.
- In-Memory: The operations in Spark are performed in-memory rather than in the database. This is what allows Spark to perform fast operations with very large quantities of data.




### RDD Transformations vs Actions

In Spark, we first create a __base RDD__ and then apply one or more transformations to that base RDD following our processing needs. Being immutable means, **once an RDD is created, it cannot be changed**. As a result, **each transformation of an RDD creates a new RDD**. Finally, we can apply one or more **actions** to the RDDs. Spark uses lazy evaluation, so transformations are not actually executed until an action occurs.


<img src="./images/rdd_diagram.png" width=500>

### Transformations

Transformations create a new dataset from an existing one by passing each dataset element through a function and returning a new RDD representing the results. In short, creating an RDD from an existing RDD is ‘transformation’.
All transformations in Spark are lazy. They do not compute their results right away. Instead, they just remember the transformations applied to some base dataset (e.g. a file). The transformations are only computed when an action requires a result that needs to be returned to the driver program.
A transformation is an RDD that returns another RDD, like map, flatMap, filter, reduceByKey, join, cogroup, etc.

### Actions
Actions return final results of RDD computations. Actions trigger execution using lineage graph to load the data into original RDD and carry out all intermediate transformations and return the final results to the driver program or writes it out to the file system. An action returns a value (to a Spark driver - the user program).

Here are some key transformations and actions that we will explore.


| Transformations   | Actions       |
|-------------------|---------------|
| map(func)         | reduce(func)  |
| filter(func)      | collect()     |
| groupByKey()      | count()       |
| reduceByKey(func) | first()       |
| mapValues(func)   | take()        |
| sample()          | countByKey()  |
| distinct()        | foreach(func) |
| sortByKey()       |               |


Let's see how transformations and actions work through a simple example. In this example, we will perform several actions and transformations on RDDs in order to obtain a better understanding of Spark processing. 

### Create a Python collection 

We need some data to start experimenting with RDDs. Let's create some sample data and see how RDDs handle it. To practice working with RDDs, we're going to use a simple Python list.

- Create a Python list `data` of integers between 1 and 1000 using the `range()` function. 
- Sanity check: confirm the length of the list (it should be 1000)

In [1]:
data = list(range(1,1001))
len(data)

# 1000

### Initialize an RDD

When using Spark to make computations, datasets are treated as lists of entries. Those lists are split into different partitions across different cores or different computers. Each list of data held in memory is a partition of the RDD. The reason why Spark is able to make computations far faster than other big data processing languages is that it allows all data to be stored __in-memory__, which allows for easy access to the data and, in turn, high-speed processing. Here is an example of how the alphabet might be split into different RDDs and held across a distributed collection of nodes:

<img src ="./images/partitions_1.png" width ="500">  
To initialize an RDD, first import `pyspark` and then create a SparkContext assigned to the variable `sc`. Use `'local[*]'` as the master.

In [2]:
import pyspark
sc = pyspark.SparkContext('local[*]')

Once you've created the SparkContext, you can use the `.parallelize()` method to create an RDD that will distribute the list of numbers across multiple cores. Here, create one called `rdd` with 10 partitions using `data` as the collection you are parallelizing.

In [3]:
rdd = sc.parallelize(data, numSlices=10)
print(type(rdd))
# <class 'pyspark.rdd.RDD'>

<class 'pyspark.rdd.RDD'>


Determine how many partitions are being used with this RDD with the `.getNumPartitions()` method.

In [4]:
rdd.getNumPartitions()
# 10

10

### Basic descriptive RDD actions

Let's perform some basic operations on our RDD. In the cell below, use the methods:
* `count`: returns the total count of items in the RDD 
* `first`: returns the first item in the RDD
* `take`: returns the first `n` items in the RDD
* `top`: returns the top `n` items
* `collect`: returns everything from your RDD


It's important to note that in a big data context, calling the collect method will often take a very long time to execute and should be handled with care!

In [5]:
# count
rdd.count()

1000

In [6]:
# first
rdd.first()

1

In [7]:
# take
rdd.take(10)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [8]:
# top
rdd.top(10)

[1000, 999, 998, 997, 996, 995, 994, 993, 992, 991]

In [9]:
# collect
rdd.collect()

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

## Map functions

Now that you've been working a little bit with RDDs, let's make this a little more interesting. Imagine you're running a hot new e-commerce startup called BuyStuff, and you're trying to track of how much it charges customers from each item sold. In the next cell, we're going to create simulated data by multiplying the values 1-1000 with a random number from 0-1.

In [10]:
import random
import numpy as np

nums = np.array(range(1, 1001))
sales_figures = data * np.random.rand(1000)
sales_figures

array([2.79503546e-01, 1.50685618e+00, 2.87941337e-01, 1.74422685e+00,
       4.53189047e+00, 3.06262908e+00, 2.51761400e+00, 1.79103251e+00,
       7.81772722e+00, 3.16105716e+00, 8.32832368e+00, 8.72080632e+00,
       6.78680036e+00, 9.15059905e+00, 6.32036308e+00, 4.52840266e+00,
       2.86025396e+00, 1.42631388e+01, 2.70274660e-01, 3.45123335e+00,
       3.67901832e+00, 4.85201842e+00, 5.82809236e+00, 1.51930935e+01,
       2.27122831e+01, 1.93412427e+01, 2.46355475e+01, 1.53017693e+01,
       3.93993643e+00, 8.03442786e+00, 4.79123891e+00, 2.91152602e+00,
       3.46150004e+00, 9.42791935e+00, 2.63038452e+01, 1.06153832e+01,
       2.33224410e+01, 2.61100203e+01, 2.14830611e+01, 4.46714016e+00,
       3.21666447e+01, 2.80210211e+01, 2.03310528e-01, 9.30919332e+00,
       2.75028680e+01, 1.47758317e+01, 3.56045614e+01, 2.67856487e+01,
       2.90265991e+01, 2.87767149e+01, 3.24571818e+01, 2.38062666e+01,
       3.23057619e+01, 1.58265112e+01, 5.02697870e+01, 1.10725797e+01,
      

We now have sales prices for 1000 items currently for sale at BuyStuff. Now create an RDD called `price_items` using the newly created data with 10 slices. After you create it, use one of the basic actions to see what's in the RDD.

In [11]:
price_items = sc.parallelize(sales_figures, numSlices=10)
price_items.take(4)

[0.2795035457605468,
 1.5068561827624427,
 0.2879413368592595,
 1.7442268485910573]

Now let's perform some operations on this simple dataset. To begin with, create a function that will take into account how much money BuyStuff will receive after sales tax has been applied (assume a sales tax of 8%). To make this happen, create a function called `sales_tax()` that returns the amount of money our company will receive after the sales tax has been applied. The function will have this parameter:

* `item`: (float) number to be multiplied by the sales tax.


Apply that function to the rdd by using the `.map()` method and assign it to a variable `renenue_minus_tax`

In [12]:
def sales_tax(num):
    return num * 0.92

revenue_minus_tax = price_items.map(sales_tax)

Remember, Spark has __lazy evaluation__, which means that the `sales_tax()` function is a transformer that is not executed until you call an action. Use one of the collection methods to execute the transformer now a part of the RDD and observe the contents of the `revenue_minus_tax` rdd.

In [13]:
# perform action to retrieve rdd values
revenue_minus_tax.take(10)

[0.25714326209970306,
 1.3863076881414473,
 0.26490602991051876,
 1.6046887007037727,
 4.169339232348058,
 2.817618752005013,
 2.3162048773452444,
 1.6477499063524523,
 7.1923090451045075,
 2.9081725886190477]

### Lambda Functions

Note that you can also use lambda functions if you want to quickly perform simple operations on data without creating a function. Let's assume that BuyStuff has also decided to offer a 10% discount on all of their items on the pre-tax amounts of each item. Use a lambda function within a `.map()` method to apply the additional 10% loss in revenue for BuyStuff and assign the transformed RDD to a new RDD called `discounted`.

In [14]:
discounted = revenue_minus_tax.map(lambda x : x*0.9)

In [15]:
discounted.take(10)

[0.23142893588973276,
 1.2476769193273027,
 0.23841542691946688,
 1.4442198306333955,
 3.7524053091132523,
 2.5358568768045116,
 2.08458438961072,
 1.4829749157172072,
 6.4730781405940565,
 2.617355329757143]

## Chaining Methods

You are also able to chain methods together with Spark. In one line, remove the tax and discount from the revenue of BuyStuff and use a collection method to see the 15 costliest items.

In [16]:
price_items.map(sales_tax).map(lambda x : x*0.9).top(15)

[805.5378710574238,
 799.2824297876743,
 744.3751858211725,
 733.6649339865309,
 722.9478284264745,
 716.4832965105293,
 697.0905761449627,
 681.7364641232641,
 681.169335796522,
 680.4814315669578,
 678.7591111078555,
 676.2710622645901,
 669.3477275123633,
 662.9687855679016,
 662.4537086564579]

## RDD Lineage


We are able to see the full lineage of all the operations that have been performed on an RDD by using the `RDD.toDebugString()` method. As your transformations become more complex, you are encouraged to call this method to get a better understanding of the dependencies between RDDs. Try calling it on the `discounted` RDD to see what RDDs it is dependent on.

In [17]:
discounted.toDebugString()

b'(10) PythonRDD[10] at RDD at PythonRDD.scala:53 []\n |   ParallelCollectionRDD[5] at readRDDFromFile at PythonRDD.scala:262 []'

### Map vs. Flatmap

Depending on how you want your data to be outputted, you might want to use `.flatMap()` rather than a simple `.map()`. Let's take a look at how it performs operations versus the standard map. Let's say we wanted to maintain the original amount BuyStuff receives for each item as well as the new amount after the tax and discount are applied. Create a map function that will return a tuple with (original price, post-discount price).

In [18]:
mapped = price_items.map(lambda x: (x, x*0.92 *0.9))
print(mapped.count())
print(mapped.take(10))

1000
[(0.2795035457605468, 0.23142893588973276), (1.5068561827624427, 1.2476769193273027), (0.2879413368592595, 0.23841542691946688), (1.7442268485910573, 1.4442198306333955), (4.531890469943541, 3.7524053091132523), (3.062629078266318, 2.5358568768045116), (2.517613997114396, 2.08458438961072), (1.7910325069048394, 1.4829749157172072), (7.8177272229396815, 6.4730781405940565), (3.161057161542443, 2.617355329757143)]


Note that we have 1000 tuples created to our specification. Let's take a look at how `.flatMap()` differs in its implementation. Use the `.flatMap()` method with the same function you created above.

In [19]:
flat_mapped = price_items.flatMap(lambda x : (x, x*0.92*0.9 ))
print(flat_mapped.count())
print(flat_mapped.take(10))

2000
[0.2795035457605468, 0.23142893588973276, 1.5068561827624427, 1.2476769193273027, 0.2879413368592595, 0.23841542691946688, 1.7442268485910573, 1.4442198306333955, 4.531890469943541, 3.7524053091132523]


Rather than being represented by tuples, all of the  values are now on the same level. When we are trying to combine different items together, it is sometimes necessary to use `.flatMap()` rather than `.map()` in order to properly reduce to our specifications. This is not one of those instances, but in the upcoming lab, you just might have to use it.

## Filter
After meeting with some external consultants, BuyStuff has determined that its business will be more profitable if it focuses on higher ticket items. Now, use the `.filter()` method to select items that bring in more than $300 after tax and discount have been removed. A filter method is a specialized form of a map function that only returns the items that match a certain criterion. In the cell below:
* use a lambda function within a `.filter()` method to meet the consultant's suggestion's specifications. set `RDD = selected_items`
* calculate the total number of items remaining in BuyStuff's inventory

In [20]:
# use the filter function
selected_items = discounted.filter(lambda x: x>300)
# calculate total remaining in inventory 
selected_items.count()

264

## Reduce

Reduce functions are where you are in some way combing all of the variables that you have mapped out. Here is an example of how a reduce function works when the task is to sum all values:

<img src = "./images/reduce_function.png" width = "600">  


As you can see, the operation is performed within each partition first, after which, the results of the computations in each partition are combined to come up with one final answer.  

Now it's time to figure out how much money BuyStuff would make from selling one of all of its items after they've reduced their inventory. Use the `.reduce()` method with a lambda function to add up all of the values in the RDD. Your lambda function should have two variables. 

In [21]:
selected_items.reduce(lambda x,y: x + y)

121634.34609686706

The time has come for BuyStuff to open up shop and start selling its goods. It only has one of each item, but it's allowing 50 lucky users to buy as many items as they want while they remain in stock. Within seconds, BuyStuff is sold out. Below, you'll find the sales data in an RDD with tuples of (user, item bought).

In [22]:
import random
random.seed(42)
# generating simulated users that have bought each item
sales_data = selected_items.map(lambda x: (random.randint(1, 50), x))

sales_data.take(7)

[(30, 313.3819807734219),
 (18, 307.1120879147867),
 (14, 309.52412489096287),
 (40, 308.0059861493359),
 (3, 318.11877618276657),
 (21, 310.6353290162177),
 (35, 338.0482254017317)]

It's time to determine some basic statistics about BuyStuff users.

Let's start off by creating an RDD that determines how much each user spent in total.
To do this we can use a method called `.reduceByKey()` to perform reducing operations while grouping by keys. After you have calculated the total, use the `.sortBy()` method on the RDD to rank the users from the highest spending to the least spending. 

In [23]:
# calculate how much each user spent
total_spent = sales_data.reduceByKey(lambda x, y: x + y)
total_spent.take(10)

[(30, 2872.4784132258205),
 (10, 1938.1610109437122),
 (40, 2304.100695446982),
 (50, 2489.7831030375896),
 (20, 3329.5146193014093),
 (41, 3393.951139060709),
 (1, 1768.3866867799934),
 (31, 2671.753756379648),
 (11, 3574.1614979481565),
 (21, 2239.743849281508)]

In [24]:
# sort the users from highest to lowest spenders
total_spent.sortBy(lambda x: x[1],ascending = False).collect()

[(16, 4201.5144619861785),
 (36, 4147.213551222092),
 (32, 3907.07454451564),
 (26, 3837.158675520832),
 (24, 3637.308885212457),
 (11, 3574.1614979481565),
 (15, 3483.1692820121298),
 (43, 3469.681658226835),
 (41, 3393.951139060709),
 (38, 3364.7861189410687),
 (20, 3329.5146193014093),
 (28, 3321.9101703270626),
 (35, 3181.4394060858344),
 (22, 3022.330894051599),
 (45, 3022.033012573248),
 (30, 2872.4784132258205),
 (4, 2855.4594073193243),
 (31, 2671.753756379648),
 (18, 2650.262145504726),
 (12, 2576.079354209214),
 (50, 2489.7831030375896),
 (39, 2481.7973794285194),
 (42, 2422.669170444958),
 (27, 2364.6228903173014),
 (44, 2346.756642674957),
 (40, 2304.100695446982),
 (21, 2239.743849281508),
 (3, 2218.5653831310365),
 (9, 2209.6674176089437),
 (33, 2183.0061212991),
 (46, 2008.5797991900297),
 (29, 2005.1659718842639),
 (25, 1989.0276522654262),
 (19, 1980.284312527896),
 (5, 1971.163608160956),
 (10, 1938.1610109437122),
 (49, 1811.0167377756159),
 (1, 1768.3866867799934),


Next, let's determine how many items were bought per user. This can be solved in one line using an RDD method. After you've counted the total number of items bought per person, sort the users from most number of items bought to least number of items. Time to start a customer loyalty program!

In [25]:
total_items = sales_data.countByKey()
sorted(total_items.items(),key=lambda kv:kv[1],reverse=True)

[(30, 9),
 (18, 9),
 (25, 9),
 (24, 9),
 (22, 8),
 (13, 8),
 (29, 8),
 (14, 8),
 (2, 8),
 (46, 7),
 (31, 7),
 (19, 7),
 (7, 7),
 (44, 7),
 (48, 7),
 (26, 7),
 (34, 6),
 (23, 6),
 (45, 6),
 (16, 6),
 (10, 6),
 (33, 6),
 (32, 5),
 (39, 5),
 (1, 5),
 (20, 5),
 (8, 5),
 (38, 5),
 (17, 5),
 (40, 4),
 (50, 4),
 (36, 4),
 (37, 4),
 (15, 4),
 (9, 4),
 (42, 4),
 (35, 4),
 (5, 4),
 (4, 4),
 (11, 4),
 (21, 3),
 (12, 3),
 (41, 3),
 (27, 3),
 (47, 3),
 (6, 2),
 (28, 2),
 (49, 2),
 (43, 2),
 (3, 1)]

### Additional Reading

- [The original paper on RDDs](https://cs.stanford.edu/~matei/papers/2012/nsdi_spark.pdf)
- [RDDs in Apache Spark](https://data-flair.training/blogs/create-rdds-in-apache-spark/)
- [Programming with RDDs](https://runawayhorse001.github.io/LearningApacheSpark/rdd.html)
- [RDD Transformations and Actions Summary](https://www.analyticsvidhya.com/blog/2016/10/using-pyspark-to-perform-transformations-and-actions-on-rdd/)

## Summary

In this lab we went through a brief introduction to RDD creation from a Python collection, setting a number of logical partitions for an RDD and extracting lineage. We also used transformations and actions to perform calculations across RDDs on a distributed setup. In the next lab, you'll get the chance to apply these transformations on different books to calculate word counts and various statistics.
